# Pydantic

파이썬 데이터 검증 라이브러리

In [13]:
# BaseModel: 데이터 검증용 클래스를 만들기 위한 부모 클래스
from pydantic import BaseModel

data = {
    "id": 1,
    "name": "neo",
    "is_active": True
}

# 일반적으로 이런 데이터들이 정상적으로 들어가 있는지 확인하려면 복잡하게 하드코딩을 해야 함

# pydantic 이용시
class User(BaseModel):
    id: int
    name: str
    is_active: bool = True
    
u1 = User(id=1, name='neo')
print(u1)  # id, name 잘 들어가고, is_active는 기본값도 True로 설정됨

u2 = User(id='2', name='smith')  # id에 str를 넣으면 에러가 난다, 변환 가능한 숫자면 알아서 변환도 해줌
u2.model_dump()  # python 객체(id=2, name=smith, is_active=True)를 한번에 딕셔너리로 변환도 해준다(나중에 JSON으로 변환하기 위함)

id=1 name='neo' is_active=True


{'id': 2, 'name': 'smith', 'is_active': True}

In [17]:
# Field: 추가 제약조건을 거는 함수
from pydantic import Field

class Order(BaseModel):
    quantity: int = Field(gt=0, description='주문 수량 0 초과')  # gt: greater than, 초과
    price: float = Field(ge=100, description='최소 단가는 100원 이상')  # ge: greater equal, 이상
    

o1 = Order(quantity=3, price=300)
# o2 = Order(quantity=-1, price=30)  # ValidationError: 2 validation errors for Order

In [ ]:
# field_validator: 더 복잡한 제약조건
from pydantic import field_validator

class Account(BaseModel):
    username: str
    password: str 
    
    # password에 다음 함수를 적용
    @field_validator('password')
    def check_password_length(cls, v):
        if len(v) < 8:  # v: value, password의 value의 길이가 8자 미만이면 에러를 내겠다는 것
            raise ValueError('비밀번호는 8자 이상이어야 합니다')  # raise: 에러를 내다
        return v

a = Account(username='bob', password='short')

# 답변의 형태를 이걸로 고정하고, LLM에게 이대로 답변을 달라고 넘기면 validation을 통과한 답변만 돌려줌

ValidationError: 1 validation error for Account
password
  Value error, 비밀번호는 8자 이상이어야 합니다 [type=value_error, input_value='short', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error